Starting Jupyter
```
python3 -m venv env
source env/bin/activate
jupyter lab
```

# Dreambooth
Some key things to change:
- `"project_name"`: Change project name to whatever you want. Avoid spaces for everything.
- `"max_training_steps"`: 2000-4000 is a good range. Choose around 2000 if you have 100 images, and around 4000 if you have 200 images.
- `"class_word"`: Keep as anthro. Or feral. Or... whatever kind of thing you're doing. But it should be extremely simple, and something the model already recognizes easily. Only matters if using regularization images.
- `"token"`: Pick a longer, made-up word the model has never seen before. Can really be anything, but make it something like "blaiddwolfyfromeldenring". "Charname+universe" is easier to remember, imo. This is what you will type in to activate it, so... don't make it insane.
- `"base_model"`: Change to whichever model you want to train on top of

In [ ]:
# Training

# This is just the name for the training run, not the token.
project_name = "Sticker Style"

# MAX STEPS
max_training_steps = 6000

# Match class_word to the category of the regularization images you chose above.
class_word = "anthro" # Anthro or feral are the best choices.

# This is the unique token you are teaching the stable diffusion model.
token = "Sticker Style" #Can be anything, but pick something rare and easy to remember/type.

dreambooth_folder = "dreambooth"
base_model = "https://sexy.canine.wf/file/yiffy-ckpt/yiffy-e18.ckpt"

# Owncloud Access
from config import *
#!pip3 install -r requirements.txt

# Used Libs
import os
import sys
import requests


## Build Environment

In [ ]:
# Download Repository and cd into it

#from git import Repo
#repository = "https://github.com/JoePenna/Dreambooth-Stable-Diffusion"
#path = "Dreambooth-Stable-Diffusion_"
#Repo.clone_from(repository, path)
if not os.path.exists(dreambooth_folder):
    !git clone https://github.com/JoePenna/Dreambooth-Stable-Diffusion {dreambooth_folder}
    %cd {dreambooth_folder}
    !git checkout 605faeaa2f0656bf31ab79749fb5d96b3ca2e69f
    !mkdir -p ./training_images
else:
    print("Dreambooth already exists")
    %cd {dreambooth_folder}

In [ ]:
# Install necessities
!pip3 install omegaconf
!pip3 install einops
!pip3 install pytorch-lightning==1.6.5
!pip3 install test-tube
!pip3 install transformers
!pip3 install kornia
!pip3 install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
!pip3 install -e git+https://github.com/openai/CLIP.git@main#egg=clip
!pip3 install setuptools==59.5.0
!pip3 install pillow==9.0.1
!pip3 install torchmetrics==0.6.0
!pip3 install -e .
!pip3 install protobuf==3.20.1
#!pip3 install gdown
!pip3 install -qq diffusers["training"]==0.3.0 transformers ftfy
!pip3 install -qq "ipywidgets>=7,<8"
!pip3 install huggingface_hub
!pip3 install ipywidgets==7.7.1
!pip3 install captionizer==1.0.1
!pip3 install pyocclient
#!pip3 install torch==1.12.1+cu116 torchvision==0.13.1+cu116 --extra-index-url https://download.pytorch.org/whl/cu116

## Download the model to train on top of

In [ ]:
# Download Base Model
link = base_model
file_name = "model.ckpt"
if not os.path.exists(file_name):
    with open(file_name, "wb") as f:
        print("Downloading %s" % link)
        response = requests.get(link, stream=True)
        total_length = response.headers.get('content-length')

        if total_length is None: # no content length header
            f.write(response.content)
        else:
            dl = 0
            total_length = int(total_length)
            for data in response.iter_content(chunk_size=40960):
                dl += len(data)
                f.write(data)
                done = int(50 * dl / total_length)
                sys.stdout.write("\r[%s%s]" % ('=' * done, ' ' * (50-done)) )    
                sys.stdout.flush()
else:
    print("Model already exists")

### Training data
#### Upload your training images

Upload 100-200 images of your character to

```
{dreambooth_folder}/training_images
```

Be sure to upload an *even* amount of images, otherwise the training inexplicably stops at 1500 steps.

#### The images should be:

- Solos only!
- Square.
- 512x512.
- Cropped to contain the most recognizable part of your character.
- Don't worry about legs/arms being cropped off, the most important part is the head. The model will make up the rest.
- A variety of angles and shots. The model can become overfitted if you only do frontal shots, for instance.
- If your character has very distinctive features or markings you want to preserve, make sure they show up in the data often.
- Variations of the same image (alt versions/edits) are fine, but don't do too many of the same.
- DON'T do black and white or uncolored sketches.
- Get as close as possible to the output you'd like to see.

A good idea is to use imagemagick for automated cropping. No, it's not the fancy kind that uses algorithms to focus on faces, but it totally works.

You're mostly on your own for cropping, but here's some...
#### Hints for cropping commands:
Windows:

`magick *.* -gravity North -crop 1:1 -resize 512x512 -format png ..\raw1\image-%d.png`

Linux:

`find -maxdepth 1 -type f -execdir magick '{}' -gravity North -crop 1:1 -resize 512x512 ../ready1/'{}.png' \;`

https://imagemagick.org/script/download.php


#### Use BIRME, it's by far the best option for bulk crop + resize:
https://www.birme.net/?target_width=512&target_height=512&rename=ORIGINAL-NAME_&border_width=-1

# Training

## This is where the magic happens.
Do not worry. It will stop at exactly the number of steps you set, even if the progress bar there looks otherwise. The estimate is usually wildly wrong. There is a separate internal counter that is not shown.

In [ ]:
# Training
#%cd ./Dreambooth-Stable-Diffusion

dataset="dataset1"
img_data_root = "./training_images/"
reg_data_root = "./regularization_images/" + dataset

#!rm -rf training_images/.ipynb_checkpoints

import os
training_images_folder = "./training_images/.ipynb_checkpoints"
if os.path.exists(training_images_folder):
    os.remove(training_images_folder)

if "uses_reg" in locals():
    !python3 "main.py" \
    --base configs/stable-diffusion/v1-finetune_unfrozen.yaml \
    -t \
    --actual_resume "model.ckpt" \
    --reg_data_root "{reg_data_root}" \
    -n "{project_name}" \
    --gpus 0, \
    --data_root "{img_data_root}" \
    --max_training_steps {max_training_steps} \
    --class_word "{class_word}" \
    --token "{token}" \
    --no-test
else:
    !python3 "main.py" \
    --base configs/stable-diffusion/v1-finetune_unfrozen.yaml \
    -t \
    --actual_resume "model.ckpt" \
    -n "{project_name}" \
    --gpus 0, \
    --data_root "{img_data_root}" \
    --max_training_steps {max_training_steps} \
    --token "{token}" \
    --no-test

## Copy and name the checkpoint file

Also prints the sha256sum, very helpful for verification/sharing your dreambooth model publicly. Take note of it.

In [ ]:
# Copy the checkpoint into the `trained_models` folder

directory_paths = !ls -d logs/*
last_checkpoint_file = directory_paths[-1] + "/checkpoints/last.ckpt"
training_images = !find training_images/*
date_string = !date +"%Y-%m-%dT%H-%M-%S"
file_name = date_string[-1] + "_" + project_name + "_" + str(len(training_images)) + "_training_images_" +  str(max_training_steps) + "_max_training_steps_" + token + "_token_" + class_word + "_class_word.ckpt"

file_name = file_name.replace(" ", "_")

!mkdir -p trained_models
!mv "{last_checkpoint_file}" "trained_models/{file_name}"
!sha256sum "trained_models/{file_name}"

print("Download your trained model file from trained_models/" + file_name + " and use in your favorite Stable Diffusion repo!")

In [ ]:
import owncloud
import os

trained_models_dir = "/workspace/Dreambooth-Stable-Diffusion/trained_models/n"
oc = owncloud.Client('http://nx.lla.im')
oc.login(oc_user, oc_pw)

for file_path, directories, files in os.walk(trained_models_dir):
    for file in files:
        if file.endswith('.ckpt'):
            full_path = os.path.join(file_path, file)
            print(f"Uploading: {full_path}")
            oc.drop_file(full_path)
 

## Upload checkpoint directly to Pixeldrain
Saves you time and money if you have a slow download speed. Slightly risky, but for some, very worth it. If your rented pod has a fast internet connection, it will upload to pixeldrain very quickly.

### It won't show any output while it is uploading. But if it worked, it will print out a link at at the bottom that you can use to download/share your model.

Take extra care, this method can go wrong in any number of ways.

In [ ]:
!wget https://github.com/ManuelReschke/go-pd/releases/download/v1.2.1/go-pd_1.2.1_linux_amd64.tar.gz -O ./go-pd.tar.gz
!tar -xf ./go-pd.tar.gz go-pd
!./go-pd upload "/workspace/Dreambooth-Stable-Diffusion/trained_models/{file_name}"